# Bash Command Generation Agent System

This notebook implements a multi-agent system to generate bash commands from natural language instructions. It uses a supervisor agent to manage two worker agents: a Generator and a Validator.

![Agent System Diagram](./img/bash_agent_system.png)

Let's start by setting up our environment and importing the necessary libraries.

In [4]:
!pip install langgraph langchain langchain_openai pydantic python-dotenv

In [5]:
import os
from dotenv import load_dotenv
from typing import Annotated, Sequence, TypedDict, Literal
import operator
import functools

from langchain_core.messages import HumanMessage, BaseMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent
from langchain.tools import StructuredTool
from langgraph.graph import StateGraph, END
from pydantic import BaseModel

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

TypeError: str expected, not NoneType

## Define Agent Tools and Functions

In [ ]:
def generate_bash_command(task_description: str) -> str:
    """Generate a bash command based on the task description."""
    return f"Generated bash command for: {task_description}"

def validate_bash_command(command: str) -> bool:
    """Validate the generated bash command."""
    # In a real scenario, you might want to implement actual validation logic
    return True

generate_tool = StructuredTool.from_function(
    func=generate_bash_command,
    name="generate_bash_command",
    description="Generate a bash command based on a task description"
)

validate_tool = StructuredTool.from_function(
    func=validate_bash_command,
    name="validate_bash_command",
    description="Validate the generated bash command"
)

## Create Agents

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

generator_agent = create_openai_functions_agent(
    llm,
    [generate_tool],
    "You are an expert in generating bash commands from natural language instructions."
)

validator_agent = create_openai_functions_agent(
    llm,
    [validate_tool],
    "You are an expert in validating bash commands for correctness and safety."
)

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

generator_node = functools.partial(agent_node, agent=generator_agent, name="Generator")
validator_node = functools.partial(agent_node, agent=validator_agent, name="Validator")

## Create Supervisor Agent

In [ ]:
members = ["Generator", "Validator"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
options = ["FINISH"] + members

class RouteResponse(BaseModel):
    next: Literal[tuple(options)]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

def supervisor_agent(state):
    supervisor_chain = (
        prompt
        | llm.with_structured_output(RouteResponse)
    )
    return supervisor_chain.invoke(state)

## Construct Graph

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

workflow = StateGraph(AgentState)

workflow.add_node("Generator", generator_node)
workflow.add_node("Validator", validator_node)
workflow.add_node("supervisor", supervisor_agent)

for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.set_entry_point("supervisor")

graph = workflow.compile()

## Test the Agent System

In [ ]:
def process_task(task_description):
    print(f"Processing task: {task_description}")
    for s in graph.stream(
        {"messages": [HumanMessage(content=task_description)]},
        {"recursion_limit": 10},
    ):
        if "__end__" not in s:
            print(s)
            print("----")
    print("Task completed.\n")

# Test with a few sample tasks
sample_tasks = [
    "List all files in the current directory",
    "Create a new directory named 'test_folder'",
    "Count the number of lines in a file named 'example.txt'"
]

for task in sample_tasks:
    process_task(task)

## Process Tasks from File

In [ ]:
import json

def process_tasks_from_file(file_path):
    with open(file_path, 'r') as f:
        tasks = f.readlines()
    
    results = {}
    for i, task in enumerate(tasks, 1):
        task = task.strip()
        print(f"Processing task {i}/{len(tasks)}: {task}")
        
        final_state = graph.invoke({"messages": [HumanMessage(content=task)]})
        generated_command = final_state['messages'][-1].content
        
        results[str(i)] = json.dumps({"invocation": task, "cmd": generated_command})
        
    with open("data/new-nl2bash-data.json", "w") as f:
        json.dump(results, f, indent=4)
    
    print(f"Processed {len(tasks)} tasks. Results saved to 'data/new-nl2bash-data.json'")

# Process tasks from the file
process_tasks_from_file("data/finetune.txt")